[Reference](https://shantun.medium.com/how-to-integrate-apache-pulsar-with-fastapi-in-python-6a43beb3c7d7)

# Step 1: Setting Up Apache Pulsar

In [1]:
pip install pulsar-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 15.1 MB/s eta 0:00:00


# Step 2: Creating a Pulsar Producer

In [2]:
from pulsar import Client

class PulsarProducer:
    def __init__(self, service_url, topic):
        self.client = Client(service_url)
        self.producer = self.client.create_producer(topic)

    def send_message(self, message):
        self.producer.send_async(message.encode('utf-8'), self.send_callback)

    def send_callback(self, message_id, exception):
        if exception is not None:
            print('Failed to send message:', exception)
        else:
            print('Message sent successfully:', message_id)

    def close(self):
        self.producer.close()
        self.client.close()

# Step 3: Creating a Pulsar Consumer

In [3]:
from pulsar import Client

class PulsarConsumer:
    def __init__(self, service_url, topic, subscription_name, message_handler):
        self.client = Client(service_url)
        self.consumer = self.client.subscribe(topic, subscription_name)
        self.message_handler = message_handler

    def start_consuming(self):
        while True:
            msg = self.consumer.receive()
            try:
                self.message_handler(msg.data().decode('utf-8'))
                self.consumer.acknowledge(msg)
            except Exception as e:
                self.consumer.negative_acknowledge(msg)
                print('Failed to process message:', e)

    def close(self):
        self.consumer.close()
        self.client.close()

# Step 4: Integrating with FastAPI

In [4]:
from fastapi import FastAPI
from pulsar_producer import PulsarProducer
from pulsar_consumer import PulsarConsumer

app = FastAPI()

@app.post('/publish/{topic}')
async def publish_message(topic: str, message: str):
    producer = PulsarProducer('pulsar://localhost:6650', topic)
    producer.send_message(message)
    producer.close()
    return {'message': 'Message published successfully'}

@app.get('/consume/{topic}/{subscription_name}')
async def consume_messages(topic: str, subscription_name: str):
    def handle_message(message):
        print('Received message:', message)

    consumer = PulsarConsumer('pulsar://localhost:6650', topic, subscription_name, handle_message)
    consumer.start_consuming()
    # This is a blocking call, so it will not return until the consumer is closed.

if __name__ == '__main__':
    uvicorn.run(app, host='0.0.0.0', port=8000)

# Step 5: Running the Application
```
uvicorn main:app
```